In [478]:
import requests 
import re as regex
import json
import pprint as pp # pretty print

import aries_cloudagent.wallet.crypto as crypto
import aries_cloudagent.wallet.util as util
import base64
import base58
import uuid

agent1 = {}
agent2 = {}
plugin = {}

## Enter Invite Url:

In [479]:
agent1['url'] = "http://agent2.localhost?c_i=eyJAdHlwZSI6ICJkaWQ6c292OkJ6Q2JzTlloTXJqSGlxWkRUVUFTSGc7c3BlYy9jb25uZWN0aW9ucy8xLjAvaW52aXRhdGlvbiIsICJAaWQiOiAiNDk0NTRiODYtZjc4NS00YzQxLWFkODAtMjcyNWUwYzAyMGFiIiwgInNlcnZpY2VFbmRwb2ludCI6ICJodHRwOi8vYWdlbnQyLmxvY2FsaG9zdCIsICJyZWNpcGllbnRLZXlzIjogWyJFVFBMUGNxMXBNdzdSMVJ5UW45NVZQZUM1RFk1VXpGYW1HSEFLQkZ3c1J4OCJdLCAibGFiZWwiOiAiQ2xpZW50IChhZG1pbikifQ=="

In [480]:
# Process invite url, delete white spaces
agent1['url'] = agent1['url'].replace(" ", "")
# Regex(substitution) to extract only the invite string from url
agent1['invite_string_b64'] = regex.sub(
           r".*(c\_i\=)", 
           r"", 
           agent1['url'])
# Decoding invite string using base64 decoder
agent1['invite_string'] = base64.b64decode(agent1['invite_string_b64'])
# Converting our invite json string into a dictionary 
agent1['invite'] = json.loads(agent1['invite_string'])

print("Decoded invitation:\n")
pp.pprint(agent1['invite'])

Decoded invitation:

{'@id': '49454b86-f785-4c41-ad80-2725e0c020ab',
 '@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation',
 'label': 'Client (admin)',
 'recipientKeys': ['ETPLPcq1pMw7R1RyQn95VPeC5DY5UzFamGHAKBFwsRx8'],
 'serviceEndpoint': 'http://agent2.localhost'}


In [481]:
### Generating a did key using aries crypto module
# Create a key pair with random seed 
# Order: 0 - public key, 1 - secret / pritvate key
plugin['keypair'] = crypto.create_keypair()
# it seems to me that did is an encoded sub string of public key
# or maybe just some arbitrary random number
did = plugin['keypair'][0][:16]
# final version of our did
plugin['did'] = base58.b58encode(did).decode("ascii")
# encoding keys to base58
plugin['public_key_b58'] = \
    base58.b58encode(plugin['keypair'][0]).decode("ascii")
plugin['private_key_b58'] = \
    base58.b58encode(plugin['keypair'][1]).decode("ascii")

# print(base58.b58encode(did))
# print(plugin['did'])
print("Private key: \n", plugin['private_key_b58'])
print("\nPublic key: \n", plugin['public_key_b58'])

Private key: 
 3y47gwwUTd6kauCRoKPNFErs7PMwRNGCQkVzHnQV765UfYemftVscviKpVZA3RsFN1BqJSCpwUcUw8YhPyWL6Pqh

Public key: 
 4A7J1pn1SSrtmysErvLxVbyn5qh4NtQFtn4fS7DacfHy


**Sending a connection request to acapy**

In [482]:
uniqueId = str(uuid.uuid4())
# print(uniqueId)

# our request body
message = {
        "@id":  uniqueId,
        "~transport": {
          "return_route": "all"
        },
        "@type": "did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/request",
        "label": "Plugin",
        "connection": {
          "DID": plugin['did'],
          "DIDDoc": {
            "@context": "https://w3id.org/did/v1",
            "id": plugin['did'],
            "publicKey": [{
              "id": plugin['did'] + "#keys-1",
              "type": "Ed25519VerificationKey2018",
              "controller": plugin['did'],
              "publicKeyBase58": plugin['public_key_b58']
            }],
            "service": [{
              "id": plugin['did'] + ";indy",
              "type": "IndyAgent",
              "recipientKeys": plugin['public_key_b58'],
              "serviceEndpoint": ""
            }]
          }
        }
      }

**Encoding/packing the request**

In [483]:
# encoding it with aries crypto function using the key that was
# given to us by aca-py in recipientKeys
decodedAcapyKey = base58.b58decode(agent1['invite']['recipientKeys'][0])
ourPrivateKey = plugin['keypair'][1]
# print(plugin['did'])
encodedMessage = \
    crypto.encode_pack_message(json.dumps(message), [decodedAcapyKey], ourPrivateKey)

encodedMessage = encodedMessage.decode("ascii")
# print("Encoded message: \n")
# pp.pprint(encodedMessage)

In [484]:
connectionRequestResponse = requests.post(agent1['invite']['serviceEndpoint'], data=encodedMessage)
assert connectionRequestResponse.text != "", "invalid response from acapy"

**Response from aca-py**

In [485]:
# a bit of a hack to simplify message unpacking,
# decode_pack_message needs a callable object for some reason
def unpackMessage(message, privateKey):
    class FindKey:
        def __init__(self, key):
            self.key = key

        def __call__(self, argument):
            return self.key

    find_key = FindKey(privateKey)   
    return crypto.decode_pack_message(message, find_key)
    
connectionRequestResponseUnpacked = \
    unpackMessage(connectionRequestResponse.text, plugin['keypair'][1])

connectionRequestResponseDict = json.loads(connectionRequestResponseUnpacked[0])

print("Decoded acapy response: \n")
pp.pprint(connectionRequestResponseUnpacked)

Decoded acapy response: 

('{"@type": "did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/response", '
 '"@id": "6f5a48d2-0014-4dd4-b798-ccd8183af6b3", "~thread": {"thid": '
 '"5367a3d9-0904-41d0-9500-114cfba64ae7"}, "connection~sig": {"@type": '
 '"did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/signature/1.0/ed25519Sha512_single", '
 '"signature": '
 '"mMkZgUytDPnDCTZCNhkMRyjteU2etXu4d6cCZiXRoV6C4Ow-VIjB3Ima9yuwvhFIJQSF2_cCa25A0ncb8um-Dw==", '
 '"sig_data": '
 '"AAAAAF69IU17IkRJRCI6ICJHZk1FWWdCSFdlYXA5WTFUTkJrV3MzIiwgIkRJRERvYyI6IHsiQGNvbnRleHQiOiAiaHR0cHM6Ly93M2lkLm9yZy9kaWQvdjEiLCAiaWQiOiAiZGlkOnNvdjpHZk1FWWdCSFdlYXA5WTFUTkJrV3MzIiwgInB1YmxpY0tleSI6IFt7ImlkIjogImRpZDpzb3Y6R2ZNRVlnQkhXZWFwOVkxVE5Ca1dzMyMxIiwgInR5cGUiOiAiRWQyNTUxOVZlcmlmaWNhdGlvbktleTIwMTgiLCAiY29udHJvbGxlciI6ICJkaWQ6c292OkdmTUVZZ0JIV2VhcDlZMVROQmtXczMiLCAicHVibGljS2V5QmFzZTU4IjogIjlZNWhWM3dIeFRadjlITWFKSEhLQW1CRXptY0tMWDNSMkdvNFRObk5FM2I0In1dLCAiYXV0aGVudGljYXRpb24iOiBbeyJ0eXBlIjogIkVkMjU1MTlTaWduYXR1cmVBdXRoZW50aWNhdGlvbjIwMT

**Unpacking connection data embedded in the response**

In [486]:
sig_data_raw = connectionRequestResponseDict['connection~sig']['sig_data']
sig_data_raw = base64.b64decode(sig_data_raw)
## TODO: replace - _ characters with some random number etc.
# avoid first 8 characters as they are a time signature
sig_data_raw = sig_data_raw[8:]
sig_data_raw = sig_data_raw.decode('ascii')

sig_data = json.loads(sig_data_raw)
pp.pprint(sig_data)

{'DID': 'GfMEYgBHWeap9Y1TNBkWs3',
 'DIDDoc': {'@context': 'https://w3id.org/did/v1',
            'authentication': [{'publicKey': 'did:sov:GfMEYgBHWeap9Y1TNBkWs3#1',
                                'type': 'Ed25519SignatureAuthentication2018'}],
            'id': 'did:sov:GfMEYgBHWeap9Y1TNBkWs3',
            'publicKey': [{'controller': 'did:sov:GfMEYgBHWeap9Y1TNBkWs3',
                           'id': 'did:sov:GfMEYgBHWeap9Y1TNBkWs3#1',
                           'publicKeyBase58': '9Y5hV3wHxTZv9HMaJHHKAmBEzmcKLX3R2Go4TNnNE3b4',
                           'type': 'Ed25519VerificationKey2018'}],
            'service': [{'id': 'did:sov:GfMEYgBHWeap9Y1TNBkWs3;indy',
                         'priority': 0,
                         'recipientKeys': ['9Y5hV3wHxTZv9HMaJHHKAmBEzmcKLX3R2Go4TNnNE3b4'],
                         'serviceEndpoint': 'http://agent2.localhost',
                         'type': 'IndyAgent'}]}}


**Adding the connection to connection list**

In [487]:
connections = [
    {
        "label": agent1['invite']['label'],
        "DIDDoc": sig_data['DIDDoc'],
        "myKey": plugin['keypair'][1]
    }
]

def packMessage(message, connection):
    # pass in our private key and recipient key to the encode_pack_message
    decodedRecipientKey = base58.b58decode(connection['DIDDoc']['service'][0]['recipientKeys'][0])
    packedMessage = \
    crypto.encode_pack_message(json.dumps(message), \
                               [decodedRecipientKey], \
                               connection['myKey'])
    return packedMessage.decode('ascii')


pp.pprint(connections)

[{'DIDDoc': {'@context': 'https://w3id.org/did/v1',
             'authentication': [{'publicKey': 'did:sov:GfMEYgBHWeap9Y1TNBkWs3#1',
                                 'type': 'Ed25519SignatureAuthentication2018'}],
             'id': 'did:sov:GfMEYgBHWeap9Y1TNBkWs3',
             'publicKey': [{'controller': 'did:sov:GfMEYgBHWeap9Y1TNBkWs3',
                            'id': 'did:sov:GfMEYgBHWeap9Y1TNBkWs3#1',
                            'publicKeyBase58': '9Y5hV3wHxTZv9HMaJHHKAmBEzmcKLX3R2Go4TNnNE3b4',
                            'type': 'Ed25519VerificationKey2018'}],
             'service': [{'id': 'did:sov:GfMEYgBHWeap9Y1TNBkWs3;indy',
                          'priority': 0,
                          'recipientKeys': ['9Y5hV3wHxTZv9HMaJHHKAmBEzmcKLX3R2Go4TNnNE3b4'],
                          'serviceEndpoint': 'http://agent2.localhost',
                          'type': 'IndyAgent'}]},
  'label': 'Client (admin)',
  'myKey': b'\x94_b\x1a\t\x1bsX\xcfBK\xf01q\x9d\x80E\x8bF6a\xfeC\xf

### FEATURE DISCOVERY

In [488]:
message = {}
message['@id'] = str(uuid.uuid4())
message['@type'] = "did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/discover-features/1.0/query"
message['~transport'] = {}
message['~transport']['return_route'] = 'all'
message['query'] = '*'
encodedMessage = packMessage(message, connections[0])
# print(encodedMessage)

endpoint = connections[0]['DIDDoc']['service'][0]['serviceEndpoint']
response = requests.post(endpoint, data=encodedMessage)
# print(response.text)

responseDecoded = unpackMessage(response.text, plugin['keypair'][1])
responseDecoded = json.loads(responseDecoded[0])
pp.pprint(responseDecoded)

{'@id': 'a44ea213-0af5-4d0a-a51a-d4d53afb96a9',
 '@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/discover-features/1.0/disclose',
 'protocols': [{'pid': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/admin-schemas/0.1'},
               {'pid': 'https://didcomm.org/introduction-service/0.1'},
               {'pid': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/basicmessage/1.0'},
               {'pid': 'https://didcomm.org/notification/1.0'},
               {'pid': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/admin-holder/0.1'},
               {'pid': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/notification/1.0'},
               {'pid': 'https://github.com/hyperledger/aries-toolbox/tree/master/docs/admin-payments/0.1'},
               {'pid': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/admin-dids/0.1'},
               {'pid': 'https://didcomm.org/routing/1.0'},
               {'pid': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/admin-credential-definitions/0.1'},
               {'pid': 'https://github.com/hyperledger/aries-toolbox/tree/

## Dids

In [489]:
message = {}
message['@id'] = str(uuid.uuid4())
message['@type'] = "did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/admin-dids/0.1/get-list-dids"
message['~transport'] = {}
message['~transport']['return_route'] = 'all'
encodedMessage = packMessage(message, connections[0])
# print(encodedMessage)

endpoint = connections[0]['DIDDoc']['service'][0]['serviceEndpoint']
response = requests.post(endpoint, data=encodedMessage)
# print(response.text)

responseDecoded = unpackMessage(response.text, plugin['keypair'][1])
responseDecoded = json.loads(responseDecoded[0])
pp.pprint(responseDecoded)

{'@id': '153dd576-4345-4da6-8173-489e2b3edd91',
 '@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/admin-dids/0.1/list-dids',
 'result': [{'did': 'SRypyC6EGLfDQrXMkLB78s',
             'verkey': 'Erv25DwntLu9Wrv9qDDpTyT5geAP2EFJKhoCEaa59wy7'},
            {'did': 'R7jMcbdYSWDBZ6iWqsTbXP',
             'verkey': 'E9Md2W85gvBbLM471Bok4PAm8skfZwHQJxEW33u2P4GL'},
            {'did': 'GfMEYgBHWeap9Y1TNBkWs3',
             'verkey': '9Y5hV3wHxTZv9HMaJHHKAmBEzmcKLX3R2Go4TNnNE3b4'},
            {'did': 'LcHB9CYg1ubfTfZfqgioMc',
             'verkey': 'BgrdPGjy34iF6HpX3ADondV8Dc92T7BkHv7CCuYwwVSY'},
            {'did': 'GA5cdEpPbyxcWyLmUGZqNX',
             'verkey': '9G8SPcRmoaepYaLKY1UhzZF1EjXe7vmKcHhVMmJmRgj5'},
            {'did': 'UuzotrWcFJ7Prz3LmRGJbB',
             'verkey': 'GDQgHUQvL827rwJt9GNCzRyfUFUASQuhr8294NDjHfiR'},
            {'did': 'A16AZUPifEcvmYjhUdyJQm',
             'verkey': '5uZGKL4EhHvre87MJKwyRHUoYt5ZZuRg1YrXMNkCUGMZ'},
            {'did': '35G1tpBSY1uXYuDUU3eFmp',
      

## Public did

In [490]:
message['@id'] = str(uuid.uuid4())
message['@type'] = "did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/admin-dids/0.1/get-public-did"
message['~transport'] = {}
message['~transport']['return_route'] = 'all'
encodedMessage = packMessage(message, connections[0])
# print(encodedMessage)

endpoint = connections[0]['DIDDoc']['service'][0]['serviceEndpoint']
response = requests.post(endpoint, data=encodedMessage)
# print(response.text)

responseDecoded = unpackMessage(response.text, plugin['keypair'][1])
responseDecoded = json.loads(responseDecoded[0])
pp.pprint(responseDecoded)

{'@id': 'b63f9f8e-3f27-4efd-a6ff-21665c38bafc',
 '@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/admin-dids/0.1/did',
 '~thread': {'thid': '6e2bd98f-1485-4245-9b94-619239a5cab1'}}


In [491]:

message = {}
message['@id'] = str(uuid.uuid4())
message['@type'] = "did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/acapy-plugin/1.0/query"
message['~transport'] = {}
message['~transport']['return_route'] = 'all'
message['query'] = '*'
# message['comment'] = "memes"
encodedMessage = packMessage(message, connections[0])
# print(encodedMessage)

endpoint = connections[0]['DIDDoc']['service'][0]['serviceEndpoint']
response = requests.post(endpoint, data=encodedMessage)
# print(response.text)

responseDecoded = unpackMessage(response.text, plugin['keypair'][1])
responseDecoded = json.loads(responseDecoded[0])
pp.pprint(responseDecoded)

{'@id': '23697ad0-c78f-46ac-be9b-59f0aa319f34',
 '@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/acapy-plugin/1.0/query',
 'comment': 'testing',
 '~thread': {'thid': '7e3f2aa9-9f17-4a2d-977a-6216d2b39546'}}
